# Reset the Knowledge Graph

## Imports

### Import python packages

To start, load some useful python packages, including some great stuff from langchain.
The "shared" notebook also sets up global constants like the connection to Neo4j,
embedding model to use, and the LLM to use for chat.


In [ ]:
%run 'shared.ipynb'

## Set up Neo4j and Langchain

### Prepare a knowledge graph interface

You can use the Langchain `Neo4jGraph` interface to send queries
to the Knowledge Graph.

In [ ]:
# Create a knowledge graph using Langchain's Neo4j integration.
# This will be used for direct querying of the knowledge graph. 
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)
kg.query("RETURN 'Hello, World!' AS message") # Test the connection by running a simple query

# Clean up the graph to remove any existing data and indexes

Use these queries to reset the current graph to a blank state,
with no indexes, contraints or data.

In [ ]:
for constraint in kg.query('SHOW CONSTRAINTS'):
    print(f"Removing constraint {constraint['name']}:")
    kg.query(f"DROP CONSTRAINT {constraint['name']}")

In [ ]:
for index in kg.query('SHOW INDEXES'):
    print(f"Removing index {index['name']}:")
    kg.query(f"""
        DROP INDEX `{index['name']}`
    """)

In [ ]:
# Remove all data by matching any node, then "detach deleting" it,
# which means removing the node and all its relationships.
kg.query("""
        MATCH (all)
        DETACH DELETE all
""")

## Basic Cypher Queries

### Script - helpful way to show schema

In [ ]:
def show_schema(kg):
  kg.refresh_schema()
  print(kg.schema)

show_schema(kg)